In [18]:
import numpy
import cv2

In [19]:
cap = cv2.VideoCapture("vtest.avi")

In [20]:
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*"XVID")
out = cv2.VideoWriter("output.avi", fourcc, 30, (1280,720))

ret, frame1 = cap.read()
ret, frame2 = cap.read()

while cap.isOpened():
    if not ret:
        break
        
    diff = cv2.absdiff(frame1, frame2)
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    
    _, thresh = cv2.threshold(blur, 50, 255, cv2.THRESH_BINARY)
    dilated = cv2.dilate(thresh, None, iterations=3)
    contours, _ = cv2.findContours(dilated.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)        
    
    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)
        
        if cv2.contourArea(contour) < 900:
            continue
            
        cv2.rectangle(frame1,(x,y), (x+w, y+h), (0, 255, 0), 2) 
        cv2.putText(frame1, "Status: {}".format("Movement"), (10,20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        
    image = cv2.resize(frame1, (1280,720))
    out.write(image)
    cv2.imshow("feed", frame1)
#     cv2.imshow("thresh", thresh)
#     cv2.imshow("dilated", dilated)

    frame1 = frame2

    ret, frame2 = cap.read()#read
        
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break   

cap.release()#close camera
out.release()#close when write
cv2.destroyAllWindows()#close window